In [1]:
import requests
import pandas as pd
from bs4 import BeautifulSoup

response = requests.get('https://raw.communitydragon.org/latest/plugins/rcp-be-lol-game-data/global/default/v1/champions/')
response.raise_for_status()
soup = BeautifulSoup(response.text, 'html.parser')
links = []
for a_tag in soup.find_all('a', href=True):
    href = a_tag['href']
    if href.endswith('.json') and not href =='-1.json':
        links.append(f"https://raw.communitydragon.org/latest/plugins/rcp-be-lol-game-data/global/default/v1/champions/{href}")

def json_extract(obj, key):
    arr = []

    def extract(obj, arr, key):
        if isinstance(obj, dict):
            for k, v in obj.items():
                if k == key:
                    arr.append(v)
                elif isinstance(v, (dict,list)):
                    extract(v, arr, key)
        elif isinstance(obj, list):
            for item in obj:
                extract(item, arr, key)

        return arr
    
    values = extract(obj, arr, key)
    return values

champion_names = []
champion_ids = []

for url in links:
    champion_json = requests.get(url).json()
    
    id = json_extract(champion_json, 'id')
    name = json_extract(champion_json, 'name')
    champion_names.append(name[0])
    champion_ids.append(id[0])

championId_dict = dict(map(lambda i, j: (int(i), j), champion_ids, champion_names))

In [2]:
import pandas as pd
tester_df = pd.read_csv('test.csv')
#another_df = tester_df.copy()

In [3]:
tester_df

,outcome,redTeam,blueTeam,Role,redTeam_tier,redTeam_win_rate,redTeam_rank,redTeam_pick_rate,redTeam_ban_rate,blueTeam_tier,blueTeam_win_rate,blueTeam_rank,blueTeam_pick_rate,blueTeam_ban_rate
0,1,Kennen,Udyr,TOP,ATier,50.69,38.98,2.4,1.6,STier,51.88,5.88,3.4,2.3
1,1,Rengar,Nidalee,JUNGLE,DTier,48.61,88.24,2.7,5.4,S+Tier,50.17,50.98,9.2,12.7
2,1,Zed,Galio,MIDDLE,DTier,48.80,87.93,7.0,18.7,S+Tier,52.42,8.62,3.8,1.5
3,1,Kai'Sa,Ezreal,BOTTOM,S+Tier,50.80,29.63,33.7,12.4,DTier,49.56,74.07,25.2,15.1
4,1,Rakan,Seraphine,UTILITY,ATier,50.11,45.45,5.0,0.8,ATier,50.63,34.09,4.7,1.1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13355,1,Rengar,Darius,TOP,DTier,48.61,88.24,2.7,5.4,S+Tier,50.55,42.37,7.4,18.3
13356,1,Hecarim,Udyr,JUNGLE,BTier,49.90,60.78,5.3,3.7,STier,51.88,5.88,3.4,2.3
13357,1,Syndra,LeBlanc,MIDDLE,CTier,49.48,68.97,5.1,2.3,STier,49.97,56.90,9.5,24.1
13358,1,Ashe,Samira,BOTTOM,S+Tier,51.05,25.93,10.5,13.6,DTier,48.69,77.78,4.9,10.7


In [4]:
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import LabelEncoder

tiers = ['S+Tier', 'STier', 'ATier', 'BTier', 'CTier', 'DTier']

tier_label_encoder = LabelEncoder()

tier_label_encoder.fit(tiers)

LabelEncoder()

In [5]:
all_champions = []
for champ in championId_dict.values():
    all_champions.append(champ)
label_encoder = LabelEncoder()
label_encoder.fit(all_champions)

roles = ['TOP', 'JUNGLE', 'MIDDLE', 'BOTTOM', 'UTILITY']
onehot_encoder = OneHotEncoder(sparse_output=False)
onehot_encoder.fit([[role] for role in roles])

# Fit and transform the data
tester_df['redTeam'] = label_encoder.transform(tester_df['redTeam'])
tester_df['redTeam_tier'] = tier_label_encoder.transform(tester_df['redTeam_tier'])

tester_df['blueTeam'] = label_encoder.transform(tester_df['blueTeam'])
tester_df['blueTeam_tier'] = tier_label_encoder.transform(tester_df['blueTeam_tier'])

#tester_df['Role'] = onehot_encoder.transform(tester_df[['Role']])
x = tester_df.drop(columns=['outcome', 'Role'])
y = tester_df['outcome']

In [6]:
import pandas as pd
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.3, random_state=42)

In [9]:
import xgboost as xgb
import numpy as np
from sklearn.metrics import accuracy_score

xgb_model = xgb.XGBClassifier(eval_metric='logloss')
xgb_model.fit(X_train, y_train)

# Make predictions
y_pred = xgb_model.predict(X_test)
#y_pred = xgb_model.predict(t)
#y_pred_prob = xgb_model.predict_proba(X_test)[:, 1] 
#y_pred_prob = xgb_model.predict_proba(t)[:, 1] 
#y_pred_percentage = y_pred_prob * 100

# Evaluate the model
accuracy = accuracy_score(y_test, y_pred)
print(f"Accuracy: {accuracy}")

Accuracy: 0.54
99.597015
[0 1 1 1 1 0 1 1 0 1 1 1 0 1 1 1 1 0 1 1 1 1 0 0 0 0 0 0 1 1 1 1 0 0 1 1 1
 1 1 1 1 0 1 1 0 0 1 1 0 1 0 0 0 0 0 1 1 0 0 0 1 1 1 0 1 1 1 0 0 1 1 1 0 0
 0 1 0 0 1 1 0 1 1 0 1 0 1 0 1 1 0 0 0 0 1 0 0 1 1 1 0 1 1 1 0 1 1 1 0 0 1
 0 0 1 0 0 0 0 0 1 1 0 1 0 1 0 1 1 0 0 0 0 1 1 0 0 0 1 0 0 1 0 0 1 0 0 1 1
 0 1]
